Use the Advanced Optimization benchmark from AC hugging face (https://huggingface.co/spaces/AccelerationConsortium/crabnet-hyperparameter)
- Optimize (minimize) y1
    - If y1 is greater than 0.2, the result is considered "bad" no matter how good the other values are
- Transfer learning: 
    - higher fidelity means more expensive computation 
    - treat fidelity1 = 0.5 as "source' and fidelity1 = 1.0 as "target"
- Multi-task: 
    - since y1 and y2 are correlated 
    - treat y1 as task1, then y2 as task 2

In [1]:
from baybe import Campaign
from baybe.objectives import SingleTargetObjective
from baybe.parameters import NumericalContinuousParameter, CategoricalParameter
from baybe.searchspace import SearchSpace
from baybe.targets import NumericalTarget
from baybe.constraints import ContinuousLinearInequalityConstraint
import numpy as np
import pandas as pd
import torch
# load the Advanced Optimization from AC huggingface
from gradio_client import Client
client = Client("AccelerationConsortium/crabnet-hyperparameter")

from baybe.utils.random import set_random_seed
# set_random_seed(36) 

# seed = 104 for x19 < x20 constraint error
# seed = 188 for x6+x15 <= 1.0 constraint error

Loaded as API: https://accelerationconsortium-crabnet-hyperparameter.hf.space ✔


In [2]:
# # define the function 
# # fix x1, x10, x19, x20
# def adv_opt(c1, c2, c3, x2, x3, x4, x5, x6, x7, x8, x9, x11, x12, x13, x14, x15, x16, x17, x18): 
#     result = client.predict(
#     	0.669938, # float (numeric value between 0.0 and 1.0) in 'x1' Slider component
# 		x2,	# float (numeric value between 0.0 and 1.0)	in 'x2' Slider component
# 		x3,	# float (numeric value between 0.0 and 1.0) in 'x3' Slider component
# 		x4,	# float (numeric value between 0.0 and 1.0) in 'x4' Slider component
# 		x5,	# float (numeric value between 0.0 and 1.0) in 'x5' Slider component
# 		x6,	# float (numeric value between 0.0 and 1.0) in 'x6' Slider component
# 		x7,	# float (numeric value between 0.0 and 1.0) in 'x7' Slider component
# 		x8,	# float (numeric value between 0.0 and 1.0) in 'x8' Slider component
# 		x9,	# float (numeric value between 0.0 and 1.0) in 'x9' Slider component
# 		0.5291,	# float (numeric value between 0.0 and 1.0) in 'x10' Slider component
# 		x11,	# float (numeric value between 0.0 and 1.0) in 'x11' Slider component
# 		x12,	# float (numeric value between 0.0 and 1.0000000000000002) in 'x12' Slider component
# 		x13,	# float (numeric value between 0.0 and 1.0) in 'x13' Slider component
# 		x14,	# float (numeric value between 0.0 and 1.0) in 'x14' Slider component
# 		x15,	# float (numeric value between 0.0 and 1.0) in 'x15' Slider component
# 		x16,	# float (numeric value between 0.0 and 1.0) in 'x16' Slider component
# 		x17,	# float (numeric value between 0.0 and 1.0) in 'x17' Slider component
# 		x18,	# float (numeric value between 0.0 and 1.0) in 'x18' Slider component
# 		0.079598,	# float (numeric value between 0.0 and 0.9999999999999998) in 'x19' Slider component
# 		0.632394,	# float (numeric value between 0.0 and 0.9999999999999998) in 'x20' Slider component
# 		c1,	# Literal['c1_0', 'c1_1'] in 'c1' Radio component
# 		c2,	# Literal['c2_0', 'c2_1'] in 'c2' Radio component
# 		c3,	# Literal['c3_0', 'c3_1', 'c3_2'] in 'c3' Radio component
# 		0.5,	# float (numeric value between 0.0 and 1.0) in 'fidelity1' Slider component
# 		api_name="/predict",
#     )
#     return result['data'][0][0]			# return y1 value only


def adv_opt(c1, c2, c3, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20):
    result = client.predict(
        x1,  # float (numeric value between 0.0 and 1.0) in 'x1' Slider component
        x2,  # float (numeric value between 0.0 and 1.0) in 'x2' Slider component
        x3,  # float (numeric value between 0.0 and 1.0) in 'x3' Slider component
        x4,  # float (numeric value between 0.0 and 1.0) in 'x4' Slider component
        x5,  # float (numeric value between 0.0 and 1.0) in 'x5' Slider component
        x6,  # float (numeric value between 0.0 and 1.0) in 'x6' Slider component
        x7,  # float (numeric value between 0.0 and 1.0) in 'x7' Slider component
        x8,  # float (numeric value between 0.0 and 1.0) in 'x8' Slider component
        x9,  # float (numeric value between 0.0 and 1.0) in 'x9' Slider component
        x10, # float (numeric value between 0.0 and 1.0) in 'x10' Slider component
        x11, # float (numeric value between 0.0 and 1.0) in 'x11' Slider component
        x12, # float (numeric value between 0.0 and 1.0) in 'x12' Slider component
        x13, # float (numeric value between 0.0 and 1.0) in 'x13' Slider component
        x14, # float (numeric value between 0.0 and 1.0) in 'x14' Slider component
        x15, # float (numeric value between 0.0 and 1.0) in 'x15' Slider component
        x16, # float (numeric value between 0.0 and 1.0) in 'x16' Slider component
        x17, # float (numeric value between 0.0 and 1.0) in 'x17' Slider component
        x18, # float (numeric value between 0.0 and 1.0) in 'x18' Slider component
        x19, # float (numeric value between 0.0 and 1.0) in 'x19' Slider component
        x20, # float (numeric value between 0.0 and 1.0) in 'x20' Slider component
        c1,  # Literal['c1_0', 'c1_1'] in 'c1' Radio component
        c2,  # Literal['c2_0', 'c2_1'] in 'c2' Radio component
        c3,  # Literal['c3_0', 'c3_1', 'c3_2'] in 'c3' Radio component
        0.5,  # float (numeric value between 0.0 and 1.0) in 'fidelity1' Slider component
        api_name="/predict",
    )
    return result['data'][0][0]  # return y1 value only

WRAPPED_FUNCTION = adv_opt

In [3]:
# define and create the search space
parameters = [
    NumericalContinuousParameter(name="x1", bounds=(0.0, 1.0)), 
    NumericalContinuousParameter(name="x2", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x3", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x4", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x5", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x6", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x7", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x8", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x9", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x10", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x11", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x12", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x13", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x14", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x15", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x16", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x17", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x18", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x19", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x20", bounds=(0.0, 1.0)),
    
    # # NumericalContinuousParameter(name="x1", bounds=(0.0, 1.0)), 
    # NumericalContinuousParameter(name="x2", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x3", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x4", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x5", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x6", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x7", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x8", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x9", bounds=(0.0, 1.0)),
    # #NumericalContinuousParameter(name="x10", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x11", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x12", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x13", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x14", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x15", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x16", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x17", bounds=(0.0, 1.0)),
    # NumericalContinuousParameter(name="x18", bounds=(0.0, 1.0)),
    # # NumericalContinuousParameter(name="x19", bounds=(0.0, 1.0)),
    # # NumericalContinuousParameter(name="x20", bounds=(0.0, 1.0)),
    CategoricalParameter(name='c1', values=['c1_0', 'c1_1'], encoding="OHE"),
    CategoricalParameter(name='c2', values=['c2_0', 'c2_1'], encoding="OHE"),
    CategoricalParameter(name='c3', values=['c3_0', 'c3_1', 'c3_2'], encoding="OHE"),
]

constraints = [
    ContinuousLinearInequalityConstraint(parameters=["x19", "x20"], coefficients=[-1.0, 1.0], rhs=0.0),
    ContinuousLinearInequalityConstraint(parameters=["x6", "x15"], coefficients=[-1.0, -1.0], rhs=-1.0), 
]

searchspace = SearchSpace.from_product(parameters=parameters, constraints=constraints)

# define objective
objective = SingleTargetObjective(target=NumericalTarget(name="Target", mode="MIN"))

# create campaign
campaign = Campaign(searchspace=searchspace, objective=objective)

In [4]:
from copy import deepcopy
# random_seed_list = [17, 28, 42, 87, 99, 131, 518, 1047, 1598, 2024]
random_seed_list = [23]
round = 30

results = pd.DataFrame()
for i in range(len(random_seed_list)):
    set_random_seed(random_seed_list[i])

    # copy the campaign
    campaign_i = deepcopy(campaign)

    for k in range(round): 
        recommendation = campaign_i.recommend(batch_size=1)
        # select the numerical columns
        numerical_cols = recommendation.select_dtypes(include='number')
        # replace values less than 1e-8 with 0 in numerical columns
        numerical_cols = numerical_cols.map(lambda x: 0 if x < 1e-6 else x)
        # update the original DataFrame
        recommendation.update(numerical_cols)

        # target value are looked up via the botorch wrapper
        target_values = []
        for index, row in recommendation.iterrows():
            # print(row.to_dict())
            # print(WRAPPED_FUNCTION(**row.to_dict()))
            target_values.append(WRAPPED_FUNCTION(**row.to_dict()))

        recommendation["Target"] = target_values

        campaign_i.add_measurements(recommendation)   
    results = pd.concat([results, campaign_i.measurements])

AppError: The upstream Gradio app has raised an exception: 1 validation error for Parameterization
N
  Value error, N must be between 1 and 10 [type=value_error, input_value=10.000000000000002, input_type=float64]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [ ]:
results